In [1]:
import pandas as pd
from datetime import datetime

## Raw Data Summary

* 2 csv files found amongst Kaggle's public datasets
* X
* X
* X
* X

## Data Preparation






In [2]:
rdt = pd.read_csv('realdonaldtrump.csv')
bo = pd.read_csv('Tweets-BarackObama.csv')

In [151]:
print('obama', bo['Date'].nunique(), 'trump', rdt['date'].nunique())

obama 6768 trump 42979


#### Tweet Dataframes
* Limited to same time frames
* Standardized the dates

In [63]:
def time_limits(date):
    oldest = datetime.strptime('01/01/13', '%d/%m/%y').date()
    newest = datetime.strptime('01/01/19', '%d/%m/%y').date()
    return ((date =< newest) & (date >= oldest))


def tweets_df(file_name, date_column, date_format, tweet_column):
    data = pd.read_csv(file_name)
    data['Datetime'] = data[date_column].apply(lambda x: datetime.strptime(x[:10], date_format).date() )
    data['Words'] = data[tweet_column].apply(lambda x: x.split(" "))
    data = data[time_limits(data['Datetime'])]
    return data

In [64]:
bo_tweets = tweets_df('Tweets-BarackObama.csv',  'Date', '%Y/%m/%d', 'Tweet-text' )
rdt_tweets = tweets_df('realdonaldtrump.csv', 'date', '%Y-%m-%d',  'content' )

In [65]:
d = {'Author': [], 'Date': [], 'Text': [], 'Words': [], 'Words': [], 'Retweets': [], 'Likes': []}
tweets = pd.DataFrame(data=d)

d1 = {'Author': 'Obama', 'Date': bo_tweets['Datetime'], 'Text': bo_tweets['Tweet-text'], 
      'Words': bo_tweets['Words'], 'Retweets': bo_tweets['Retweets'], 'Likes': bo_tweets['Likes']}
df1 = pd.DataFrame(data=d1)
d2 = {'Author': 'Trump', 'Date': rdt_tweets['Datetime'], 'Text': rdt_tweets['content'], 
      'Words': rdt_tweets['Words'], 'Retweets': rdt_tweets['retweets'], 'Likes': rdt_tweets['favorites']}
df2 = pd.DataFrame(data=d2)
tweets = tweets.append(df1)
tweets = tweets.append(df2)
tweets

,Author,Date,Text,Words,Retweets,Likes
36,Obama,2018-12-29,I hope you find inspiration in the stories of ...,"[I, hope, you, find, inspiration, in, the, sto...",11579.0,92906.0
37,Obama,2018-12-29,Leaders like Jonny Boucher a Chicago native wh...,"[Leaders, like, Jonny, Boucher, a, Chicago, na...",19658.0,113752.0
38,Obama,2018-12-29,Leaders like Hong Hoang who mobilized a youth-...,"[Leaders, like, Hong, Hoang, who, mobilized, a...",3791.0,29283.0
39,Obama,2018-12-29,Leaders like Moussa Kondo and Sandor Lederer w...,"[Leaders, like, Moussa, Kondo, and, Sandor, Le...",3841.0,27248.0
40,Obama,2018-12-29,Leaders like Dejah Powell who started an organ...,"[Leaders, like, Dejah, Powell, who, started, a...",5169.0,35342.0
...,...,...,...,...,...,...
36344,Trump,2018-12-31,"....Senator Schumer, more than a year longer t...","[....Senator, Schumer,, more, than, a, year, l...",18858.0,78559.0
36345,Trump,2018-12-31,HAPPY NEW YEAR!pic.twitter.com/bHoPDPQ7G6,"[HAPPY, NEW, YEAR!pic.twitter.com/bHoPDPQ7G6]",32578.0,149581.0
36346,Trump,2018-12-31,MEXICO IS PAYING FOR THE WALL through the many...,"[MEXICO, IS, PAYING, FOR, THE, WALL, through, ...",20684.0,94748.0
36347,Trump,2018-12-31,"The Democrats will probably submit a Bill, bei...","[The, Democrats, will, probably, submit, a, Bi...",19841.0,89922.0


In [66]:
print( tweets['Date'].max(), tweets['Date'].min())

2018-12-31 2013-01-02


#### Word Dataframes
* Created two dictionaries as dataframes
* Filtered out conjunctions, articles, prepositions, and conjunctions for simplicity (chose to keep pronouns)
* Combed over capitalizations and extra symbols (e.g., both 'president' and '-president' grouped under 'president)

In [13]:
exclude = ['to', 'the', 'a', 'an', 'of', 'for', 'and', 'is', 'in', 'on', 'are', 'that', 'this', 'have', 'at', 'be', 'about', 
           'from', 'it', "it's", 'its', 'than', 'then', 'from', 'with', 'as', 'who', 'do', 'has', 'not', 'by', 'what', '@', '"', 
           '#', '&', '…', '-', 'so', 'but', 'i', 'you', 'he', 'she', 'we', 'they', 'my', 'your', 'her', 'his', 'our', 'their']

In [14]:
def words_df(data):
    words_list = []
    for i in range (0, len(data)):
        words_list += data['Words'][i]
    words_df = pd.DataFrame({'Word': words_list})
    words_df['word'] = words_df['Word'].apply(lambda x: x.lower())
    words_df['Exclude'] = words_df['word'].apply(lambda x: x in exclude)
    words_df = words_df[words_df['Exclude'] == False]
    return words_df

In [15]:
bo_words = words_df(bo_tweets)
rdt_words = words_df(rdt_tweets)

In [16]:
bo_words['word'].value_counts().head(20)

obama            2170
—president       1447
president        1208
if                499
health            478
more              463
congress          405
time              400
get               358
americans         355
will              343
can               342
make              339
care              325
all               304
up                280
people            276
watch             267
climate           267
#actonclimate     252
Name: word, dtype: int64

In [113]:
rdt_words['word'].value_counts().head(20)

realdonaldtrump    7932
will               6490
great              5787
trump              3665
was                3369
thank              3003
all                2927
just               2652
very               2461
people             2114
no                 2059
would              1972
new                1912
get                1898
if                 1801
president          1777
like               1680
should             1653
more               1574
out                1573
Name: word, dtype: int64

In [ ]:
def find_word(df, word):
    df['FindWord'] = df[Words].apply(lambda x: word in x)
    return df

def plot_word(word):
    find_word(bo_tweets, word)
    find_word(rdt_tweets, word)
    
    